# QuickStart
Based on [Pytorch Quickstart](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)

- This section runs through the API for common tasks in ML.

## Working with data

PyTorch has 2 primitives to work with data: `torch.utils.data.DataLoader` and `torch.utils.data.Dataset`.

`Dataset` stores the samples and their corresponding labels, and `Dataloader` wraps an iterable around the `Dataset`.

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch has TorchText, TorchVision and TorchAudio, all contain datasets.

We will use a TorchVision dataset.

The `torchvision.datasets` module contains `Dataset` objects for many real-world vision data like CIFAR, COCO etc

Here is the [full list](https://pytorch.org/vision/stable/datasets.html)

We'll use a 3 datasets to get the gist: FashionMNIST.

Every TorchVision Dataset includes two arguments: `transform` and `target_transform` to modify the samples and labels respectively.

In [3]:
# download training data from open datasets

train_data = datasets.FashionMNIST(
root = 'data',
download=True,
train = True,
transform=ToTensor()
) #required argument = `root` if locally available else #2nd required argument is download=True 

test_data = datasets.FashionMNIST(
root = 'data',
download=True,
train = False,
transform=ToTensor()
) #required argument = `root` if locally available else #2nd required argument is download=True 

100%|██████████| 26421880/26421880 [00:03<00:00, 8407566.64it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 67598.20it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:04<00:00, 1013919.09it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 11322641.32it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



The `Dataset` is passed as an argument to `DataLoader` to wrap an iterable over our dataset and support automatic batching, sampling, shuffling and multiprocess data loading.

A batch size of 64 means each element in the dataloader iterable will return a batch of 64 features & labels.

In [4]:
#Create dataloaders.
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} & data type : {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) & data type : torch.int64


# Creating Models

To define a NN in PyTorch, a class is created that inherits from nn. Module, the layers are defined in the network in the __init__ function and specify how data will pass through the network in the forward function. To accelerate in the NN, we move it to th GPU or MPS if available

In [5]:
device = ("cuda" if torch.cuda.is_available() else "mps" if torch.backend.mps.is_available() else "cpu")
print("Device : ", device)

Device :  cuda


In [6]:
#Define Model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(nn.Linear(28*28,512),nn.ReLU(),nn.Linear(512,512),nn.ReLU(),nn.Linear(512,10))
    
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

        

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
dummy_train_data = datasets.CIFAR10(root='data1',download=True)

In [ ]:
dummy_train_data = datasets.CIFAR100(root='data1',download=True)

In [ ]:
# dummy_train_data = datasets.Country211(root='data1',download=True)

In [ ]:
dummy_train_data = datasets.DTD(root='data1',download=True)